代码段8-1 树的抽象基类的一部分（后接代码段8-2）

In [3]:
class Tree:
    """Abstract base class representing a tree structure."""
    
    # ---------- nested Position class ----------
    class Position:
        """An abstraction representing the location of a single element."""
        def element(self):
            raise NotImplementedError('must be implemented by subclass')
        
        def __eq__(self, other):
            """Return True if other Position represents the same location."""
            raise NotImplementedError('must be implemented by subclass')
        
        def __ne__(self, other):
            """Return True if other does not represent the same location."""
            return not (self == other)    # opposite of __eq__
        
    # ---------- abstract methods that concrete subclass must support ----------
    def root(self):
        """Return Position representing the tree's root (or None if empty)."""
        raise NotImplementedError('must be implemented by subclass')
    
    def parent(self, p):
        """Return Position representing the tree's parent (or None if p is root)."""
        raise NotImplementedError('must be implemented by subclass')
    
    def num_children(self, p):
        """Return the number of children that Position p has."""
        raise NotImplementedError('must be implemented by subclass')
    
    def children(self, p):
        """Generate an iteration of Positions representing p's children."""
        raise NotImplementedError('must be implemented by subclass')
    
    def __len__(self):
        """Return the total number of elements in the tree."""
        raise NotImplementedError('must be implemented by subclass')
    
    # ---------- concrete methods implemented in this class ----------
    def is_root(self, p):
        """Return True if Position p represents the root of the tree."""
        return self.root() == p
    
    def is_leaf(self, p):
        """Return True if Position p does not have any children."""
        return self.num_children(p) == 0
    
    def is_empty(self):
        """Return True if the tree is empty."""
        return len(self) == 0

**深度**

树T中节点p的深度的定义：

p的深度同样也可以按如下递归定义：
 * 如果p是根节点，那么p的深度为0。
 * 否则，p的深度就是其父节点的深度加1。

代码段8-3 树类中计算深度的算法

In [4]:
    def depth(self, p):
        """Return the number of levels separating Position p from the root."""
        if self.is_root(p):
            return 0
        else:
            return 1 + self.depth(self.parent(p))

**高度**

树T中节点p的高度的定义如下：
 * 如果p是一个叶子节点，那么它的高度为0。
 * 否则，p的高度是它孩子节点中的最大高度加1。

命题8-4 Tree类中的方法_height1。需要注意的是，该方法调用了计算深度的算法

In [5]:
    def _height1(self):
        """Return the height of the tree."""
        return max(self.depth(p) for p in self.positions() if self.is_leaf(p))

代码段8-6 计算整个树或者一个给定位置作为根节点的子树的高度的Tree.height方法

In [6]:
    def height(self, p=None):
        """Return the height of the subtree rooted at Position p.
        
        If p is None, return the height of the entire tree.
        """
        if p is None:
            p = self.root()
        return self._height2(p)    # start _height2 recursion

**二叉树**

二叉树是具有以下属性的有序树：

1）每个节点最多有两个孩子节点。

2）每个孩子节点被命名为*左孩子*或者*右孩子*。

3）对于每个节点的孩子节点，在顺序上，左孩子先于右孩子。

若子树的根为内部节点v的左孩子或右孩子，则该子树相应地被称为节点v的*左子树*或*右子树*。

若每个节点都有零个或者两个节点，则这样的二叉树为*完全二叉树*。一些人也把这种树称为*满二叉树*。

**递归二叉树的定义**

使用递归方式定义二叉树，此时二叉树或者为空树，或者由以下条件组成：
 * 二叉树T的根为节点r，其存储一个元素。
 * 二叉树（可能为空）称为T的左子树。
 * 二叉树（可能为空）称为T的右子树。

代码段8-7 从代码段8-1和8-2已存在的Tree抽象基类中扩展的BinaryTree抽象基类

In [7]:
class BinaryTree(Tree):
    """Abstract base class representing a binary tree structure."""
    
    # ---------- addational abstract methods ----------
    def left(self, p):
        """Return a Position representing p's left child.
        
        Return None if p does not have a left child.
        """
        raise NotImplementedErrore('must be implemented by subclass')
    
    def right(self, p):
        """Return a Position representing p's right child.
        
        Return None if p does not have a right child.
        """
        raise NotImplementedErrore('must be implemented by subclass')
    
    # ---------- concrete methods implemented in this class ----------
    def sibling(self, p):
        """Return a Position representing p's sibing (or None if no sibling)."""
        parent = self.parent(p)
        if parent is None:    # p must be the root
            return None       # root has no sibling
        else:
            if p == self.left(parent):
                return self.right(parent)    # possibly None
            else:
                return self.left(parent)     # possibly None
    
    def children(self, p):
        """Generate an iteration of Position representing p's children."""
        if self.left(p) is not None:
            yield self.left(p)
        if self.right(p) is not None:
            yield self.right(p)

代码段8-8 LinkedBinaryTree类的开始（后接代码段8-9~8-11）

In [3]:
class LinkedBinaryTree(BinaryTree):
    """Linked representation of a binary tree structure."""
    
    class _Node:    # Lightweight, nonpublic class for storing a node.
        __slots__ = '_element', '_parent', '_left', '_right'
        def __init__(self, element, parent=None, left=None, right=None):
            self._element = element
            self._parent = parent
            self._left = left
            self._right = right
    
    class Position(BinaryTree.Position):
        """An abstraction representing the location of a single element."""
        
        def __init__(self, container, node):
            """Constructor should not be invoked by user."""
            self._container = container
            self._node = node
        
        def element(self):
            """Return the element stored at this Position."""
            return self._node._element
        
        def __eq__(self, other):
            """Return True if other is a Position representing the same location."""
            return type(other) is type(self) and other._node is self._node
    
    def _validate(self, p):
        """Return associated node, if position is valid."""
        if not isinstance(p, self.Position):
            raise TypeError('p must be proper Position type.')
        if p._container is not self:
            raise ValueError('p does not belong to this container')
        if p._node._parent is p._node:    # convention for deprecated nodes
            raise ValueError('p is no longer valid')
        return p._node
    
    def _make_position(self, node):
        """Return Position instance for given node (or None if no node)."""
        return self.Position(self, node) if node is not None else None

NameError: name 'BinaryTree' is not defined

代码段8-9 LinkedBinaryTree类的公开访问方法。该类从代码段8-8开始，且在代码段8-10和8-11中继续

In [4]:
    # ---------- binary tree constructor ----------
    def __init__(self):
        """Create an initially empty binary tree."""
        self._root = None
        self._size = 0
    
    # ---------- public accessors ----------
    def __len__(self):
        """Return the total number of elements in the tree."""
        return self._size
    
    def root(self):
        """Return the root Position of the tree (or None if tree is empty)."""
        return self._make_position(self._root)
    
    def parent(self, p):
        """Return the Position of p's parent(or None if p is root)."""
        node = self._validate(p)
        return self._make_position(node._parent)
    
    def left(self, p):
        """Return the Position of p's left child (or None if no left child)."""
        node = self._validate(p)
        return self._make_position(node._left)
    
    def right(self, p):
        """Return the Position of p's right (or None if no right child)."""
        node = self._validate(p)
        return self._make_position(node._right)
    
    def num_children(self, p):
        """Return the number of children of Position p."""
        node = self._validate(p)
        count = 0
        if node._left is not None:    # left child exists
            count += 1
        if node._right is not None:   # right child exists
            count += 1
        return count

代码段8-10 LinkedBinaryTree类的非公开更新方法（后接代码段8-11）

In [5]:
    def _add_root(self, e):
        """Place element e at the root of an empty tree and return new Position.

        Raise ValueError if tree nonempty.
        """
        if self._root is not None:
            raise ValueError('Root exists')
        self._size = 1
        self._root = self._Node(e)
        return self._make_position(self._root)
    
    def _add_left(self, p, e):
        """Create a new left child for Position p, storing element e.
        
        Return the Position of new node.
        Raise ValueError if Position p is invalid or already has a left child.
        """
        node = self._validate(p)
        if node._left is not None:
            raise ValueError('Left child exists')
        self._size += 1
        node._left = self._Node(e, node)    # node is its parent
        return self._make_position(node._left)
    
    def _add_right(self, p, e):
        """Create a new right child for Position p, storing element e.
        
        Return the Position of new node.
        Raise ValueError if Position p is invalid or already has a right child.
        """
        node = self._validate(p)
        if node._right is not None:
            raise ValueError('Right child exists')
        self._size += 1
        node._right = self._Node(e, node)    # node is its parent
        return self._make_position(node._right)
    
    def _replace(self, p, e):
        """Replace the element at position p with e, and return old element."""
        node = self._validate(p)
        old = node._element
        node._element = e
        return old

代码段8-11 LinkedBinaryTree类的非公开更新方法（接代码段8-10）

In [6]:
    def _delete(self, p):
        """Delete the node at Position p, and replace it with its child, if any.
        
        Return the element that had been stored at Position p.
        Raise ValueError if Position p is invalid or p has two children.
        """
        node = self._validate(p)
        if self.num_children(p) == 2:
            raise ValueError('p has two children')
        child = node._left if node._left else node._right    # might be None
        if child is not None:
            child._parent = node._parent     # child's grandparent becomes parent
        if node is self._root:
            self._root = child    # child becomes root
        else:
            parent = node._parent
            if node is parent._left:
                parent._left = child
            else:
                parent._right = child
        self._size -= 1
        node._parent = node    # convention for deprecated node
        return node._element
    
    def _attach(self, p, t1, t2):
        """Attach trees t1 and t2 as left and right subtrees of ecternal p."""
        node = self._validate(p)
        if not self.is_leaf(p):
            raise ValueError('position must be leaf')
        if not type(self) is type(t1) is type(t2):    # all 3 trees must be same type
            raise TypeError('Tree types must match')
        self._size += len(t1) + len(t2)
        if not t1.is_empty():    # attached t1 as left subtree of node
            t1._root._parent = node
            node.left = t1._root
            t1._root = None    # set t1 instance to empty
            t1._size = 0
        if not t2.is_empty():    # attached t2 as left subtree of node
            t2._root._parent = node
            node.left = t2._root
            t2._root = None    # set t2 instance to empty
            t2._size = 0